# TEST

In [13]:
!pwd
isLocalServer = True

%load_ext autoreload
%autoreload 2
%matplotlib inline
import json, sys, os

if isLocalServer is True:
  print(f'This is in the local server, therefore your folder path should be added to the system path.')
  
  sys_path_to_be_added = '/notebook/personal/ksuchoi216/FaceID-model/'
  if not sys_path_to_be_added in sys.path:
    sys.path.insert(0, sys_path_to_be_added)
    os.chdir(sys_path_to_be_added)

  print(f'System path as follows:')
  for path in sys.path:
    print(f'    {path}')

from utils import load_config
cfg = load_config('config_test-various-face-extraction.json')

/notebook/personal/ksuchoi216/FaceID-model
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
This is in the local server, therefore your folder path should be added to the system path.
System path as follows:
    /notebook/personal/ksuchoi216/FaceID-model/
    /notebook/personal/ksuchoi216/faceID-model/
    /notebook
    /root/.pyenv/versions/3.9.1/lib/python39.zip
    /root/.pyenv/versions/3.9.1/lib/python3.9
    /root/.pyenv/versions/3.9.1/lib/python3.9/lib-dynload
    
    /root/.pyenv/versions/python39/lib/python3.9/site-packages


In [14]:
import numpy as np
# loading numpy data

def loadNumpyImagesToDict(user_names_list, data_source_path):
  user_names = user_names_list
  data_source = data_source_path

  numpy_data = {}
  print(numpy_data)
  for i, user_name in enumerate(user_names):
    print(i, user_name)
    file_path = data_source + user_name + '_images.npy'
    numpy_data[user_name] = np.load(file_path, allow_pickle=True)

  print('example dimension: {}'.format(numpy_data['jhyoo'].shape))


In [16]:
test = numpy_data['jhyoo'][0]

img = Image.fromarray(test)

plt.imshow(img)
plt.show()

NameError: name 'numpy_data' is not defined

In [26]:
from PIL import Image
from deepface import DeepFace
import matplotlib.pyplot as plt

models = [
  "VGG-Face", 
  "Facenet", 
  "Facenet512", 
  "OpenFace", 
  "DeepFace", 
  "DeepID", 
  "ArcFace", 
  # "Dlib", 
  # "SFace",
]


data_source = './data/photos_from_video/'
user_names = cfg['user_names']
theNumberOfImages = cfg['theNumberOfImages']
for i, user_name in enumerate(user_names):
  for num in range(theNumberOfImages):
    img_path = data_source + user_name + '/' + str(num) + '.jpg'
    for i in range(len(models)):
      emb = DeepFace.represent(img_path=img_path, model_name = models[i])
      print(len(emb))
      
    if num > 1:
      break
  if i > 1:
    break

1/1 [==============================] - 0s 305ms/step
2622
1/1 [==============================] - 0s 94ms/step
128
1/1 [==============================] - 0s 114ms/step
512
1/1 [==============================] - 0s 34ms/step
128
1/1 [==============================] - 0s 134ms/step
4096
1/1 [==============================] - 0s 28ms/step
160
1/1 [==============================] - 0s 107ms/step
512
1/1 [==============================] - 0s 138ms/step
2622
1/1 [==============================] - 0s 75ms/step
128
1/1 [==============================] - 0s 71ms/step
512
1/1 [==============================] - 0s 38ms/step
128
1/1 [==============================] - 0s 80ms/step
4096
1/1 [==============================] - 0s 28ms/step
160
1/1 [==============================] - 0s 106ms/step
512
1/1 [==============================] - 0s 136ms/step
2622
1/1 [==============================] - 0s 78ms/step
128
1/1 [==============================] - 0s 73ms/step
512
1/1 [==============================]